In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_notebook

from PIL import Image, ImageOps
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
from six import StringIO
import tensorflow as tf
import tensorflow_hub as hub
from six.moves.urllib.request import urlopen
import os
import pickle
from multiprocessing import Pool
import pandas as pd
import operator

mod_res=1

pkl_files = os.listdir('features/')

exclude = []
for i in range(1,94):
    if i%3 != mod_res:
        exclude.append('results_dict_index_'+str(i)+'.pkl')
pkl_files = [x for x in pkl_files if x not in exclude]

results_dict = {}
for pkl_file in tqdm_notebook(pkl_files):
    input = open('features/'+pkl_file, 'rb')
    cur_dict = pickle.load(input)
    results_dict.update(cur_dict)
    input.close()

filelist = os.listdir('../input/test/')
filelist = ['../input/test/'+x for x in filelist]
JPG_TEST_LIST = filelist


JPG_INDEX_LIST = [x for x in list(results_dict.keys()) if 'index' in x]


/home/tianqi/anaconda3/envs/google-landmark/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from bisect import bisect_left
import faiss                   # make faiss available
from collections import Counter

def match_images(image_1_path, image_2_path, visual=False):
  distance_threshold = 0.8

  # Read features.
  locations_1, descriptors_1 = results_dict[image_1_path]
  num_features_1 = locations_1.shape[0]
  #print("Loaded image 1's %d features" % num_features_1)
  locations_2, descriptors_2 = results_dict[image_2_path]
  num_features_2 = locations_2.shape[0]
  #print("Loaded image 2's %d features" % num_features_2)

  # Find nearest-neighbor matches using a KD tree.
  d1_tree = cKDTree(descriptors_1)
  _, indices = d1_tree.query(
      descriptors_2, distance_upper_bound=distance_threshold)

  #print(indices)
  # Select feature locations for putative matches.
  locations_2_to_use = np.array([
      locations_2[i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])
  locations_1_to_use = np.array([
      locations_1[indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])

  # Perform geometric verification using RANSAC.
  try:
      _, inliers = ransac(
          (locations_1_to_use, locations_2_to_use),
          AffineTransform,
          min_samples=3,
          residual_threshold=20,
          max_trials=1000)
  except:
    return 0

  try:
    res = sum(inliers)
  except:
    res = 0
    
  # Visualize correspondences.
  if visual == True:
      print('Found %d inliers' % sum(inliers))  
        
      _, ax = plt.subplots(figsize=(9, 18))
      img_1 = mpimg.imread(image_1_path)
      img_2 = mpimg.imread(image_2_path)
      inlier_idxs = np.nonzero(inliers)[0]
      plot_matches(
          ax,
          img_1,
          img_2,
          locations_1_to_use,
          locations_2_to_use,
          np.column_stack((inlier_idxs, inlier_idxs)),
          matches_color='b')
      ax.axis('off')
      ax.set_title('DELF correspondences')
  return res


class Matcher(object):
    def __init__(self, img1):
        self.img1 = img1
    def __call__(self, x):
        return match_images(self.img1,
                            x,
                            False)

In [3]:

vec_idx = []
vec_idx2img = {}
vec_database = []

cur_idx = 0
for key, val in tqdm(results_dict.items()):
    if 'index' not in key:
        continue
    ftrs = val[1]
    num_ftr = ftrs.shape[0]
    for i in range(num_ftr):
        vec_idx.append(cur_idx)
        vec_database.append(ftrs[i,:])
        # only insert the last one
        if i==num_ftr-1:
            vec_idx2img[cur_idx] = key

        cur_idx += 1

d = 40
nb = len(vec_database)
xb = np.array(vec_database)
index = faiss.IndexFlatL2(d)   # build the index
print('is_trained:',index.is_trained)
index.add(xb)                  # add vectors to the index
print('total data point:',index.ntotal)

key_val_pairs = list(vec_idx2img.items())
key_val_pairs.sort(key=lambda r: r[0])
keys = [x[0] for x in key_val_pairs]

def get_img_from_idx(idx):
    return key_val_pairs[bisect_left(keys,idx)][1]


100%|██████████| 425406/425406 [00:19<00:00, 21304.06it/s]


is_trained: True
total data point: 45663841


In [4]:

def search_db(img_file,num_cand=100):
    k = 100
    min_threshold = 5
    D, I = index.search(results_dict[img_file][1], k) # sanity check
    cntr = Counter([get_img_from_idx(x) for x in I.flatten().tolist()])
    #cntr = {x : cntr[x] for x in cntr if cntr[x] >= min_threshold }
    cntr = sorted(cntr.items(), key=operator.itemgetter(1), reverse=True)[:num_cand]
    candidates = [x[0] for x in cntr]
    return candidates

In [5]:
# img_names_884 = pd.read_csv('../input/test_landmarks_884.csv',header=None)
# img_names_1919 = pd.read_csv('../input/test_landmarks_1919.csv',header=None)
# img_names_1218 = pd.read_csv('../input/test_landmarks_batch3_1218.csv',header=None)
# img_names_406 = pd.read_csv('../input/test_landmarks_batch4_406.csv',header=None)
# img_names_1111 = pd.read_csv('../input/test_landmarks_batch5_1111.csv',header=None)
img_names_500 = pd.read_csv('../input/test_landmarks_batch6_500.csv',header=None)

# img_names = list(img_names_884.values.flatten().tolist()
#                 +img_names_1919.values.flatten().tolist()
#                 +img_names_1218.values.flatten().tolist()
#                 +img_names_406.values.flatten().tolist()
#                 +img_names_1111.values.flatten().tolist())
img_names = img_names_500.values.flatten().tolist()

In [6]:
img_names = [x[:-4] for x in img_names]

In [7]:
img_names = ['../input/test/'+x+'.jpg' for x in img_names]

In [8]:
len(img_names)

500

In [9]:
img_names = [x for x in img_names if os.path.isfile(x)]

In [10]:
len(img_names)

500

In [11]:
import datetime
import warnings
warnings.filterwarnings("ignore")

sub_dict = {}
cnt_idx = 0
for img_name in tqdm_notebook(img_names):
    cnt_idx += 1
    candidates= search_db(img_name)
    try:
        with Pool(8) as pool:
            res = list(
                tqdm_notebook(pool.imap(Matcher(img_name),  candidates),
                     total=len(candidates))
            )

        this_landmark = []
        for idx in np.array(res).argsort()[-100:][::-1]:
            this_landmark.append(candidates[idx][15:])
        sub_dict[img_name] = this_landmark
        if cnt_idx % 20 == 0:
            output = open('sub_dict_{}_mod{}_batch6.pkl'.format(cnt_idx,mod_res), 
                          'wb')
            pickle.dump(sub_dict, output)
            output.close()
    except Exception as err:
        print(err)

output = open('sub_dict_{}_batch6.pkl'.format(mod_res), 
              'wb')
pickle.dump(sub_dict, output)
output.close()